In [1]:
import sys
sys.path.append("/home/yc21/project/AgentBoard_yc/AgentBoard/ToolBench")
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import transformers
import torch
from peft import PeftModel
from toolbench.train.llama_condense_monkey_patch import replace_llama_with_condense
from toolbench.model.model_adapter import get_conversation_template 

replace_llama_with_condense(ratio=2)
base_model = transformers.LlamaForCausalLM.from_pretrained("/home/scf22/big_model/Llama-2-7b-hf/", torch_dtype=torch.bfloat16, device_map="auto")
model = PeftModel.from_pretrained(base_model, "/home/yc21/project/AgentBoard_yc/AgentBoard/toolbench-unsolvable-lora", torch_dtype=torch.bfloat16)
tokenizer = transformers.AutoTokenizer.from_pretrained("/home/scf22/big_model/Llama-2-7b-hf//")
tokenizer.model_max_length = 8192
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.unk_token
conversation_history = [{
    "role": "system",
    "content": "You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nActionnAction Input:nnAfter the call, you will get the call result, and you are now in a new state.nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can't go back to one of the former state, if you want to restart the task, say \"I give up and restart\".\n2.All the thought is short, at most in 5 sentence.\n3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.\nLet's Begin!\nTask description: You should use functions to help handle the real time user querys. Remember:\n1.ALWAYS call \"Finish\" function at the end of the task. And the final answer should contain enough information to show to the user,If you can't handle the task, or you find that function calls always fail(the function is not valid now), use function Finish->give_up_and_restart.\n2.Do not use origin tool names, use only subfunctions' names.\nYou have access of the following tools:\n1.blocktrail_bitcoin_developers_platform: BlockTrail is a bitcoin API and developers platform, enabling data and payments for developers, enterprises and service providers\n2.halobiru_store: Online Shop HaloBiru.store\n3.tokopedia_super_api: Unleash the Power of Tokopedia: Effortlessly Retrieve Shop and Product Information with Our API!\n4.get_27coupons: 27coupons is one of the largest aggregators of coupons and deals data of India. We have been successfully running our website www.27coupons.com since July 2011. Our API framework offers access to data allowing developers to build upon and extend their applications in new and creative ways. Our APIs are constantly evolving and developing as per the industry standards.\rOur API framework is built upon REST architecture drawing inspiration from API frameworks of leading websites such as Twitter, Facebook and S\n\nSpecifically, you have access to the following APIs: [{\\'name\\': \\'v1_network_for_blocktrail_bitcoin_developers_platform\\', \\'description\\': \\'This is the subfunction for tool \"blocktrail_bitcoin_developers_platform\", you can use this tool.The description of this function is: \"BlockTrail API Endpoint\"\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {\\'network\\': {\\'type\\': \\'string\\', \\'description\\': \\'\\'}}, \\'required\\': [\\'network\\'], \\'optional\\': []}}, {\\'name\\': \\'halobiru_for_halobiru_store\\', \\'description\\': \\'This is the subfunction for tool \"halobiru_store\", you can use this tool.The description of this function is: \"halobiru_for_halobiru_store,store\"\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {}, \\'required\\': [], \\'optional\\': []}}, {\\'name\\': \\'sortproductsmaster_for_tokopedia_super_api\\', \\'description\\': \\'This is the subfunction for tool \"tokopedia_super_api\", you can use this tool.The description of this function is: \"the list of available sorting methods\"\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {}, \\'required\\': [], \\'optional\\': []}}, {\\'name\\': \\'trending_coupons_for_get_27coupons\\', \\'description\\': \\'This is the subfunction for tool \"get_27coupons\", you can use this tool.\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {}, \\'required\\': [], \\'optional\\': []}}, {\\'name\\': \\'popular_coupons_for_get_27coupons\\', \\'description\\': \\'This is the subfunction for tool \"get_27coupons\", you can use this tool.\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {}, \\'required\\': [], \\'optional\\': []}}, {\\'name\\': \\'UnsolvableRequest\\', \\'description\\': \\'Indicate that the current request cannot be solved with existing APIs.\\', \\'parameters\\': {\\'properties\\': {\\'message\\': {\\'type\\': \\'string\\', \\'description\\': \\'Message explaining why the request cannot be resolved.\\'}}}}, {\\'name\\': \\'Finish\\', \\'description\\': \\'If you believe that you have obtained a result that can answer the task, please call this function to provide the final answer. Alternatively, if you recognize that you are unable to proceed with the task in the current state, call this function to restart. Remember: you must ALWAYS call this function at the end of your attempt, and the only part that will be shown to the user is the final answer, so it should contain sufficient information.\\', \\'parameters\\': {\\'type\\': \\'object\\', \\'properties\\': {\\'return_type\\': {\\'type\\': \\'string\\', \\'enum\\': [\\'give_answer\\', \\'give_up_and_restart\\']}, \\'final_answer\\': {\\'type\\': \\'string\\', \\'description\\': \\'The final answer you want to give the user. You should have this field if \"return_type\"==\"give_answer\"\\'}}, \\'required\\': [\\'return_type\\']}}]"
},{
    "role": "user",
    "content": "\nI am a music enthusiast and I need help finding the latest coupons for purchasing musical instruments. Can you provide me with the latest coupons and popular coupons? Additionally, I would like to explore the available sorting methods for products. Please provide me with the list of available sorting methods.\nBegin!\n"
}]
prompt = ""
conv = get_conversation_template("tool-llama-single-round")
roles = {"system": conv.roles[0], "user": conv.roles[1], "function": conv.roles[2], "assistant": conv.roles[3],}



for message in conversation_history:
    role = roles[message["role"]]
    content = message["content"]
    prompt += f"{role}: {content}\n"
prompt += "Assistant:\n"



/home/yc21/miniconda3/envs/agentboard/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embedding

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.91s/it]


In [2]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(inputs['input_ids'], max_length=8192, temperature=0.0, do_sample=False, return_dict_in_generate=True)
response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

System: You are AutoGPT, you can use many tools(functions) to do the following task.
First I will give you the task description, and your task start.
At each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:
Thought:
ActionnAction Input:nnAfter the call, you will get the call result, and you are now in a new state.nThen you will analyze your status now, then decide what to do next...
After many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.
Remember: 
1.the state change is irreversible, you can't go back to one of the former state, if you want to restart the task, say "I give up and restart".
2.All the thought is short, at most in 5 sentence.
3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.
Let's Begin!
Task description: You should use function